<div style="background-color:#000;"><img src="pqn.png"></img></div>

We use warnings to filter messages, numpy and pandas for fast calculations, and Zipline libraries for running backtests, managing orders, simulating trading costs, creating data pipelines, and working with stock price and factor data.

In [ ]:
import warnings

In [ ]:
import numpy as np
import pandas as pd
from zipline import run_algorithm
from zipline.api import (
    attach_pipeline,
    cancel_order,
    date_rules,
    get_datetime,
    get_open_orders,
    order_target_percent,
    pipeline_output,
    schedule_function,
    set_commission,
    set_slippage,
    time_rules,
)
from zipline.finance import commission, slippage
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.factors import AverageDollarVolume, CustomFactor

In [ ]:
warnings.filterwarnings("ignore")

This code sets up a few key parameters we’ll use for our historical lookback window and to determine the size of the universe we analyze.

In [ ]:
LOOKBACK_WEEKS = 151
LOOKBACK_DAYS = LOOKBACK_WEEKS * 5
UNIVERSE_SIZE = 3000
QUANTILE = 4

These variables control how far back we look (about three years), how many stocks we consider (3000), and how we divide them into groups for ranking (quartiles).

## Build custom stock factors

Here, we define a custom factor to measure weekly volatility and build a pipeline that selects stocks with the lowest volatility from higher-volume names.

In [ ]:
class WeeklyVolatility(CustomFactor):
    """
    Computes standard deviation of weekly returns over a 3-year window.
    Weekly return is defined over 5 consecutive trading days:
        r_week = (close[t] / close[t-4]) - 1
    We take non-overlapping 5-day chunks across the window for stability.
    """

    inputs = [USEquityPricing.close]
    window_length = LOOKBACK_DAYS

    def compute(self, today, assets, out, closes):
        n_weeks = closes.shape[0] // 5
        if n_weeks < 2:  # not enough weeks to compute a standard deviation
            out[:] = np.nan
            return

        trimmed = closes[-n_weeks * 5 :, :]  # (n_weeks*5, n_assets)
        weekly_open = trimmed[::5, :]  # first close in each 5-day block
        weekly_close = trimmed[4::5, :]  # last close in each 5-day block
        weekly_rets = (weekly_close / weekly_open) - 1  # shape (n_weeks, n_assets)

        out[:] = np.nanstd(weekly_rets, axis=0, ddof=1)

In [ ]:
def make_pipeline():
    adv20 = AverageDollarVolume(window_length=20)
    base_universe = adv20.top(UNIVERSE_SIZE)

    vol = WeeklyVolatility(mask=base_universe)
    # Select lowest-volatility quartile by taking the bottom N within the ADV-filtered universe.
    target_count = max(1, UNIVERSE_SIZE // QUANTILE)
    lows = vol.bottom(target_count, mask=base_universe)

    pipe = Pipeline(
        columns={
            "adv20": adv20,
            "vol": vol,
            "low_vol_long": lows,
        },
        screen=base_universe,
    )
    return pipe

In this block, we create a special stock ranking that calculates how much a stock’s price jumps around from week to week over the last three years. We only consider stocks with good trading volume. We then sort these stocks and filter down to those with the most stable (least volatile) prices, forming our candidate list.

By defining a custom calculation for weekly volatility, we’re able to focus on stocks with lower risk. The pipeline groups our target stocks by filtering out those that don’t trade enough and then scoring what’s left, so each month we can focus on the most stable performers. This process uses Zipline’s factor modeling tools, making it easy to plug into the rest of our analysis. We return a data pipeline that tags each qualifying stock with its volume, its volatility, and a flag for inclusion in our portfolio.

## Configure and execute the trading strategy

Next, we set up our strategy to use the custom data pipeline, schedule when we rebalance, and decide which stocks to hold at each rebalance.

In [ ]:
def initialize(context):
    # Attach pipeline
    attach_pipeline(make_pipeline(), "lowvol_pipe")

    # Monthly rebalance at market open (first trading day)
    schedule_function(
        rebalance, date_rules.month_start(), time_rules.market_open(minutes=1)
    )

In [ ]:
def before_trading_start(context, data):
    # Pull latest factor data
    context.pipe = pipeline_output("lowvol_pipe")

    # Determine long basket (lowest-vol quartile)
    longs_frame = context.pipe[context.pipe["low_vol_long"]]
    context.long_assets = list(longs_frame.index)

In [ ]:
def rebalance(context, data):

    # Filter tradable and priced today
    tradable_longs = [a for a in context.long_assets if data.can_trade(a)]

    if len(tradable_longs) == 0:
        # Nothing tradable; flatten all positions
        for asset in list(context.portfolio.positions.keys()):
            if data.can_trade(asset):
                order_target_percent(asset, 0.0)
        return

    # Target equal weights across long basket (long-only)
    w = 1.0 / float(len(tradable_longs))

    # Cancel any open orders to avoid drift
    oo = get_open_orders()
    for asset, orders in oo.items():
        for o in orders:
            cancel_order(o)

    # Close positions that are no longer in the target basket
    current_pos = list(context.portfolio.positions.keys())
    for asset in current_pos:
        if asset not in tradable_longs and data.can_trade(asset):
            order_target_percent(asset, 0.0)

    # Set target weights for longs
    for asset in tradable_longs:
        order_target_percent(asset, w)

This code ties everything together for live backtesting. Each month, we update our list of low-volatility stocks and assign equal weights to each, selling anything that’s no longer in our target group. When nothing in our group can be traded that day, we exit all positions. We also make sure to clean up any outstanding orders before placing new ones to avoid unnecessary trading. Scheduling and pipeline hooks make the whole process hands-off once started, keeping our portfolio focused and up-to-date.

## Run backtest and plot results

Now, we set the backtest period, run the simulation using Quandl US price data, and graph how the strategy’s value grows over time.

In [ ]:
start = pd.Timestamp("2015")
end = pd.Timestamp("2018")

In [ ]:
perf = run_algorithm(
    start=start,
    end=end,
    initialize=initialize,
    before_trading_start=before_trading_start,
    capital_base=100_000,
    bundle="quandl",
)

In [ ]:
perf.portfolio_value.plot(title="Low Volatility Factor Equity Curve", ylabel="Strategy Equity")

Here, we run a full portfolio simulation over three years, tracking how a $100,000 account grows if we always hold the least volatile, high-volume US stocks. Zipline uses actual historical pricing from Quandl, making the results realistic for live trading. The plot gives us a clear picture of how steady or bumpy our approach might feel in the real world. By checking this chart, we quickly see if our low-volatility approach actually produced smoother and stronger returns than just holding the market.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.